# Deep Learning Project Part 3

## By: i221898 i222018 i221915

## Implementation of improved version

In [12]:
import os, sys, math, random, time, glob
from typing import Tuple, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
from torchvision import transforms as T
import timm

# -------------------------
# Configuration
# -------------------------
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# Dataset / file paths (Kermany pediatric pneumonia dataset)
DATA_ROOT = "/kaggle/input/xraydata/chest_xray"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 20
LR = 5e-5             # slightly larger LR for head-training (we'll reduce when unfreezing)
WEIGHT_DECAY = 1e-4

# Loss weights (reduced segmentation influence)
LAMBDA_SEG = 0.0      # disable seg auxiliary initially; enable small value later if desired
LAMBDA_ATT = 0.0
SAVE_DIR = "./ckpt_hybrid_deit_fixed"
os.makedirs(SAVE_DIR, exist_ok=True)

# Early stopping
PATIENCE = 5

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -------------------------
# Verify dataset
# -------------------------
def verify_dataset(root):
    ok = os.path.isdir(os.path.join(root, "train")) and os.path.isdir(os.path.join(root, "test"))
    return ok

if not verify_dataset(DATA_ROOT):
    print("Dataset not found locally at", DATA_ROOT)
    print("If running on Kaggle: add the 'Chest X-ray Images (Pneumonia)' dataset via the Notebook 'Add data' UI and re-run.")
else:
    print("Dataset found:", DATA_ROOT)

# -------------------------
# Utility: unsupervised lung mask generator (heuristic)
# -------------------------
def generate_lung_mask_orig(image_bgr: np.ndarray) -> np.ndarray:
    if image_bgr.ndim == 3:
        gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    else:
        gray = image_bgr.copy()
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(gray)
    blur = cv2.GaussianBlur(cl, (5,5), 0)
    _, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    th_inv = 255 - th
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    closed = cv2.morphologyEx(th_inv, cv2.MORPH_CLOSE, kernel, iterations=2)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel, iterations=1)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(opened, connectivity=8)
    if num_labels <= 1:
        mask = (opened > 0).astype(np.uint8)
    else:
        areas = stats[1:, cv2.CC_STAT_AREA]
        idx_sorted = np.argsort(areas)[::-1]
        mask = np.zeros_like(opened, dtype=np.uint8)
        for i in idx_sorted[:2]:
            lbl = i+1
            mask[labels == lbl] = 1
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15)))
    return mask.astype(np.uint8)

# -------------------------
# Dataset class for Kermany dataset (train/val/test)
# -------------------------
class PediatricCXRDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None, mask_transform=None, generate_mask=True):
        self.root = root_dir
        self.split = split
        self.transform = transform
        self.mask_transform = mask_transform
        self.generate_mask = generate_mask
        self.samples = []
        split_dir = os.path.join(root_dir, split)
        if not os.path.isdir(split_dir):
            raise RuntimeError(f"Split dir not found: {split_dir}")
        for label_name in ["PNEUMONIA", "NORMAL"]:
            lab_dir = os.path.join(split_dir, label_name)
            if not os.path.isdir(lab_dir):
                continue
            for p in glob.glob(os.path.join(lab_dir, "*")):
                if p.lower().endswith((".jpeg",".jpg",".png")):
                    label = 1 if label_name == "PNEUMONIA" else 0
                    self.samples.append((p, label))
        if len(self.samples) == 0:
            raise RuntimeError(f"No samples found under {split_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img_bgr = cv2.imread(path, cv2.IMREAD_COLOR)
        if img_bgr is None:
            img_bgr = np.zeros((IMG_SIZE,IMG_SIZE,3), dtype=np.uint8)
        h,w = img_bgr.shape[:2]
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        mask = generate_lung_mask_orig(img_bgr) if self.generate_mask else np.zeros((h,w),dtype=np.uint8)
        img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
        mask_resized = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        if self.transform is not None:
            from PIL import Image
            pil = Image.fromarray(img_resized)
            img_t = self.transform(pil)
        else:
            img_t = T.ToTensor()(img_resized)
        if self.mask_transform is not None:
            from PIL import Image
            pil_mask = Image.fromarray((mask_resized*255).astype(np.uint8))
            mask_t = self.mask_transform(pil_mask)
        else:
            mask_t = torch.from_numpy(mask_resized).unsqueeze(0).float()/255.0
        return img_t, torch.tensor(label, dtype=torch.long), mask_t

# -------------------------
# Prepare datasets & transforms
# -------------------------
print("Preparing datasets...")

original_train_ds = PediatricCXRDataset(DATA_ROOT, split="train", transform=None, mask_transform=None, generate_mask=True)
test_ds = PediatricCXRDataset(DATA_ROOT, split="test", transform=None, mask_transform=None, generate_mask=True)

# Stratified split indices
train_samples = original_train_ds.samples
train_labels_for_split = [label for _, label in train_samples]
train_idx, val_idx = train_test_split(
    range(len(train_samples)),
    test_size=0.15,
    stratify=train_labels_for_split,
    random_state=SEED
)

print(f"Original train size: {len(original_train_ds)}")
print(f"Train indices: {len(train_idx)}, Val indices: {len(val_idx)}")

# Transforms
train_tf = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.85, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomApply([T.ColorJitter(brightness=0.12, contrast=0.12)], p=0.5),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

test_tf = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

mask_tf = T.Compose([T.ToTensor()])

class TransformedSubset(Dataset):
    def __init__(self, dataset, indices, transform=None, mask_transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform
        self.mask_transform = mask_transform
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        img, label, mask = self.dataset[actual_idx]
        
        if self.transform:
            from PIL import Image
            img_pil = T.ToPILImage()(img)
            img = self.transform(img_pil)
        
        if self.mask_transform:
            from PIL import Image
            mask_pil = T.ToPILImage()(mask)
            mask = self.mask_transform(mask_pil)
            
        return img, label, mask

# Create train/val/test subsets
train_ds = TransformedSubset(original_train_ds, train_idx, transform=train_tf, mask_transform=mask_tf)
val_ds = TransformedSubset(original_train_ds, val_idx, transform=test_tf, mask_transform=mask_tf)
test_ds = PediatricCXRDataset(DATA_ROOT, split="test", transform=test_tf, mask_transform=mask_tf, generate_mask=True)

# -------------------------
# Weighted sampler to handle imbalance
# -------------------------
train_label_list = [original_train_ds.samples[i][1] for i in train_idx]
class_counts = np.bincount(train_label_list)
inv_freq = {i: 1.0 / class_counts[i] for i in range(len(class_counts))}
sample_weights = [inv_freq[l] for l in train_label_list]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print("Sizes: train", len(train_ds), "val", len(val_ds), "test", len(test_ds))

# -------------------------
# Calculate class weights (for loss)
# -------------------------
train_labels = [original_train_ds.samples[i][1] for i in train_idx]
class_counts = np.bincount(train_labels)
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
print(f"Class counts: {class_counts}, Class weights: {class_weights.cpu().numpy()}")

# -------------------------
# Model components with DEiT Stem
# -------------------------
class DEiTStem(nn.Module):
    def __init__(self, out_channels=128):
        super().__init__()
        self.deit = timm.create_model('deit_base_patch16_224', pretrained=True, num_classes=0)
        self.projection = nn.Conv2d(768, out_channels, 1)
        
    def forward(self, x):
        x = self.deit.forward_features(x)
        patch_tokens = x[:, 1:]
        batch_size = x.shape[0]
        features = patch_tokens.reshape(batch_size, 14, 14, 768).permute(0, 3, 1, 2)
        features = self.projection(features)
        features = F.interpolate(features, size=(56, 56), mode='bilinear', align_corners=False)
        return features

class SegHead(nn.Module):
    def __init__(self, in_ch, mid_ch=64):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Dropout2d(0.1),
            nn.Conv2d(in_ch, mid_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(mid_ch, mid_ch//2, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(mid_ch//2, 1, 1)
        )

    def forward(self, f):
        return self.decoder(f)

class SimpleTransformerEncoder(nn.Module):
    def __init__(self, d_model=128, nhead=8, num_layers=3, dim_feedforward=256, dropout=0.1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, 
            dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tokens, src_key_padding_mask=None):
        out = self.encoder(tokens, src_key_padding_mask=src_key_padding_mask)
        out = self.norm(out)
        return out

class CrossAttentionFusion(nn.Module):
    def __init__(self, d_model=128, nhead=8):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model), 
            nn.ReLU(), 
            nn.Dropout(0.1),
            nn.LayerNorm(d_model)
        )

    def forward(self, cls_tok, kv_tokens):
        out, attn_w = self.attn(query=cls_tok, key=kv_tokens, value=kv_tokens, need_weights=True)
        out = self.ffn(out.squeeze(1)).unsqueeze(1)
        return out, attn_w

class HybridLungViT(nn.Module):
    def __init__(self, num_classes=2, d_model=128, token_grid=(7,7)):
        super().__init__()
        self.stem = DEiTStem(out_channels=d_model)
        self.seghead = SegHead(in_ch=d_model, mid_ch=64)
        self.token_grid = token_grid
        self.d_model = d_model
        self.left_enc = SimpleTransformerEncoder(d_model=d_model, nhead=8, num_layers=2, dropout=0.1)
        self.right_enc = SimpleTransformerEncoder(d_model=d_model, nhead=8, num_layers=2, dropout=0.1)
        self.global_enc = SimpleTransformerEncoder(d_model=d_model, nhead=8, num_layers=2, dropout=0.1)
        self.fusion = CrossAttentionFusion(d_model=d_model, nhead=8)
        self.cls_head = nn.Sequential(
            nn.Dropout(0.3),
            nn.LayerNorm(d_model), 
            nn.Linear(d_model, num_classes)
        )
        self.cls_token = nn.Parameter(torch.randn(1,1,d_model))

    def forward(self, x, mask=None):
        B = x.size(0)
        f = self.stem(x)
        seg_logits = self.seghead(f)
        
        f_up = F.interpolate(f, size=(IMG_SIZE, IMG_SIZE), mode='bilinear', align_corners=False)
        
        left_tokens = []
        right_tokens = []
        global_tokens = []
        
        for i in range(B):
            fmap = f_up[i]
            if mask is not None:
                m = mask[i,0].detach().cpu().numpy()
                ys, xs = np.where(m>0.5)
                if len(xs) > 0:
                    cx = int(xs.mean())
                else:
                    cx = IMG_SIZE//2
            else:
                cx = IMG_SIZE//2
                
            left_map = fmap[:, :, :cx] if cx > 1 else fmap[:,:, :1]
            right_map = fmap[:, :, cx:] if cx < IMG_SIZE-1 else fmap[:,:, cx-1:cx]
            
            try:
                lt = F.adaptive_avg_pool2d(left_map.unsqueeze(0), output_size=self.token_grid).squeeze(0)
                rt = F.adaptive_avg_pool2d(right_map.unsqueeze(0), output_size=self.token_grid).squeeze(0)
            except Exception:
                lt = F.adaptive_avg_pool2d(fmap.unsqueeze(0), output_size=self.token_grid).squeeze(0)
                rt = lt.clone()
                
            lt_tokens = lt.view(self.d_model, -1).permute(1,0)
            rt_tokens = rt.view(self.d_model, -1).permute(1,0)
            
            gt = F.adaptive_avg_pool2d(fmap.unsqueeze(0), output_size=(max(1,self.token_grid[0]//2), max(1,self.token_grid[1]//2))).squeeze(0)
            gt_tokens = gt.view(self.d_model, -1).permute(1,0)
            
            left_tokens.append(lt_tokens)
            right_tokens.append(rt_tokens)
            global_tokens.append(gt_tokens)

        left_tokens = torch.stack([t for t in left_tokens], dim=0).to(x.device)
        right_tokens = torch.stack([t for t in right_tokens], dim=0).to(x.device)
        global_tokens = torch.stack([t for t in global_tokens], dim=0).to(x.device)

        left_out = self.left_enc(left_tokens)
        right_out = self.right_enc(right_tokens)
        konk = torch.cat([global_tokens, left_out, right_out], dim=1)
        global_out = self.global_enc(konk)

        cls = self.cls_token.expand(B, -1, -1).to(x.device)
        kv = torch.cat([left_out, right_out], dim=1)
        cls_fused, attn_w = self.fusion(cls, kv)
        logits = self.cls_head(cls_fused.squeeze(1))
        
        return logits, seg_logits, attn_w

# -------------------------
# Loss helpers
# -------------------------
def dice_loss(pred, target, eps=1e-6):
    pred = torch.sigmoid(pred)
    inter = (pred * target).sum(dim=[1,2,3])
    union = pred.sum(dim=[1,2,3]) + target.sum(dim=[1,2,3])
    dice = (2*inter + eps) / (union + eps)
    return 1.0 - dice.mean()

# -------------------------
# Instantiate model, optimizers
# -------------------------
model = HybridLungViT(num_classes=2, d_model=128, token_grid=(7,7)).to(DEVICE)

# Use weighted loss for imbalanced data; label smoothing if available
try:
    criterion_cls = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
except TypeError:
    criterion_cls = nn.CrossEntropyLoss(weight=class_weights)

opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=3, verbose=True)

# Mixed precision scaler
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

print("Model with DEiT stem created!")
print("Number of parameters:", sum(p.numel() for p in model.parameters()))

# -------------------------
# TTA prediction helper (simple horiz flip average)
# -------------------------
def tta_probs(model, xb, mask=None):
    """Return averaged positive-class probabilities (tensor on same device) for batch xb."""
    model.eval()
    xb = xb.to(DEVICE)
    probs_list = []
    with torch.no_grad():
        logits, _, _ = model(xb, mask)
        p0 = torch.softmax(logits, dim=1)[:,1]
        probs_list.append(p0)
        # horizontal flip TTA
        xb_flip = torch.flip(xb, dims=[3])
        # flip mask too if provided
        mask_flip = None
        if mask is not None:
            mask_flip = torch.flip(mask.to(DEVICE), dims=[3])
        logits_f, _, _ = model(xb_flip, mask_flip)
        p1 = torch.softmax(logits_f, dim=1)[:,1]
        probs_list.append(p1)
    # average
    p_avg = torch.stack(probs_list, dim=0).mean(dim=0)
    return p_avg

# -------------------------
# Enhanced evaluation function (uses TTA)
# -------------------------
def evaluate(model, loader, detailed=False, use_tta=True):
    model.eval()
    correct = 0; total = 0
    y_true = []; y_scores = []; y_pred = []
    seg_losses = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Eval"):
            xb, yb, mask = batch
            xb = xb.to(DEVICE); yb = yb.to(DEVICE); mask = mask.to(DEVICE)
            
            if use_tta:
                probs = tta_probs(model, xb, mask)
            else:
                logits, seg_logits, _ = model(xb, mask)
                probs = torch.softmax(logits, dim=1)[:,1]
            
            # For preds, threshold at 0.5 on averaged prob
            preds = (probs > 0.5).long()
            
            # compute seg loss using single forward (no TTA) to avoid extra overhead
            logits_single, seg_logits_single, _ = model(xb, mask)
            if seg_logits_single.shape != mask.shape:
                seg_logits_single = F.interpolate(seg_logits_single, size=(mask.shape[2], mask.shape[3]), mode='bilinear', align_corners=False)
            seg_losses.append(dice_loss(seg_logits_single, mask).item())
            
            correct += (preds == yb).sum().item()
            total += xb.size(0)
            y_true.extend(yb.detach().cpu().numpy().tolist())
            y_scores.extend(probs.detach().cpu().numpy().tolist())
            y_pred.extend(preds.detach().cpu().numpy().tolist())
    
    acc = 100.0 * correct / total if total > 0 else 0.0
    try:
        auc = roc_auc_score(y_true, y_scores)
    except Exception:
        auc = float('nan')
    
    if detailed:
        print("\n" + "="*50)
        print("DETAILED CLASSIFICATION REPORT:")
        print("="*50)
        print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))
        
        cm = confusion_matrix(y_true, y_pred)
        print("CONFUSION MATRIX:")
        print(cm)
        if cm.size == (2,2):
            print(f"True Negatives: {cm[0,0]}, False Positives: {cm[0,1]}")
            print(f"False Negatives: {cm[1,0]}, True Positives: {cm[1,1]}")
            specificity = cm[0,0] / (cm[0,0] + cm[0,1]) if (cm[0,0] + cm[0,1]) > 0 else 0
            sensitivity = cm[1,1] / (cm[1,0] + cm[1,1]) if (cm[1,0] + cm[1,1]) > 0 else 0
            print(f"Sensitivity/Recall: {sensitivity:.4f}, Specificity: {specificity:.4f}")
    
    return acc, auc, np.mean(seg_losses) if seg_losses else float('nan'), y_true, y_pred, y_scores

# -------------------------
# Freeze / unfreeze helpers for stage-wise training
# -------------------------
def set_stem_grad(enabled: bool):
    for p in model.stem.deit.parameters():
        p.requires_grad = enabled

# freeze stem initially
freeze_epochs = 3
set_stem_grad(False)

# -------------------------
# Training loop with early stopping
# -------------------------
train_losses = []; val_history = []
best_auc = -1.0
best_epoch = -1
no_improve = 0

print(f"Starting training with early stopping (patience={PATIENCE})...")

for epoch in range(1, EPOCHS+1):
    model.train()
    t0 = time.time()
    running_loss = 0.0
    processed = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    
    # unfreeze at specified epoch
    if epoch == freeze_epochs + 1:
        print("Unfreezing DEiT stem and lowering LR for finetuning")
        set_stem_grad(True)
        for g in opt.param_groups:
            g['lr'] = LR * 0.2  # lower LR when fine-tuning full model
    
    try:
        for batch in pbar:
            xb, yb, mask = batch
            xb = xb.to(DEVICE); yb = yb.to(DEVICE); mask = mask.to(DEVICE)
            
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits, seg_logits, attn_w = model(xb, mask)
                
                if seg_logits.shape != mask.shape:
                    seg_logits = F.interpolate(seg_logits, size=(mask.shape[2], mask.shape[3]), mode='bilinear', align_corners=False)
                
                ce = criterion_cls(logits, yb)
                bce = F.binary_cross_entropy_with_logits(seg_logits, mask)
                dloss = dice_loss(seg_logits, mask)
                seg_loss = 0.5 * bce + 0.5 * dloss
                attn_reg = torch.tensor(0.0, device=xb.device)
                loss = ce + LAMBDA_SEG * seg_loss + LAMBDA_ATT * attn_reg
            
            opt.zero_grad()
            scaler.scale(loss).backward()
            
            # Gradient clipping
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            scaler.step(opt)
            scaler.update()
            
            running_loss += loss.item() * xb.size(0)
            processed += xb.size(0)
            avg_loss = running_loss / processed if processed > 0 else 0.0
            pbar.set_postfix(loss=f"{avg_loss:.4f}", processed=processed)
            
    except KeyboardInterrupt:
        print("\nTraining interrupted by user (KeyboardInterrupt). Saving checkpoint...")
        ckpt_path = os.path.join(SAVE_DIR, f"interrupted_epoch{epoch}_ckpt.pth")
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "opt_state": opt.state_dict(),
            "scaler_state": scaler.state_dict(),
            "train_losses": train_losses,
            "val_history": val_history,
            "best_auc": best_auc
        }, ckpt_path)
        print("Checkpoint saved to", ckpt_path)
        break

    # Validation (use TTA for more stable val metrics)
    val_acc, val_auc, val_seg_loss, _, _, _ = evaluate(model, val_loader, use_tta=True)
    train_losses.append(running_loss / max(1, len(train_loader.dataset)))
    val_history.append((val_acc, val_auc, val_seg_loss))
    sched.step(val_auc)
    
    t1 = time.time()
    print(f"Epoch {epoch} finished in {t1-t0:.1f}s")
    print(f"Val Acc: {val_acc:.2f}%, Val AUC: {val_auc:.4f}, Val Seg Loss: {val_seg_loss:.4f}")
    
    # Early stopping check
    if val_auc > best_auc + 0.001:
        best_auc = val_auc
        best_epoch = epoch
        no_improve = 0
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, "best_model.pth"))
        print(f"✅ Saved best model (AUC improved to {best_auc:.4f})")
    else:
        no_improve += 1
        print(f"⏳ No improvement for {no_improve}/{PATIENCE} epochs (best AUC: {best_auc:.4f})")
    
    if no_improve >= PATIENCE:
        print(f"🛑 Early stopping triggered at epoch {epoch}!")
        break

print(f"\nTraining completed. Best val AUC: {best_auc:.4f} at epoch {best_epoch}")

# -------------------------
# Final evaluation on test set
# -------------------------
if os.path.exists(os.path.join(SAVE_DIR, "best_model.pth")):
    print("\nLoading best model for final test evaluation...")
    model.load_state_dict(torch.load(os.path.join(SAVE_DIR, "best_model.pth"), map_location=DEVICE))
    
    print("\n" + "="*60)
    print("FINAL TEST SET EVALUATION (with TTA)")
    print("="*60)
    
    test_acc, test_auc, test_seg_loss, test_true, test_pred, test_scores = evaluate(
        model, test_loader, detailed=True, use_tta=True
    )
    
    print(f"\n📊 TEST RESULTS:")
    print(f"Test Acc: {test_acc:.2f}%")
    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test Seg Loss: {test_seg_loss:.4f}")
    
    # Compare with validation performance
    print(f"\n📈 PERFORMANCE COMPARISON:")
    print(f"Best Val AUC: {best_auc:.4f}")
    print(f"Test AUC:     {test_auc:.4f}")
    print(f"Gap:          {best_auc - test_auc:.4f}")
    
print("\nDone. Model & artifacts saved under:", SAVE_DIR)

Device: cuda
Dataset found: /kaggle/input/xraydata/chest_xray
Preparing datasets...
Original train size: 5216
Train indices: 4433, Val indices: 783
Sizes: train 4433 val 783 test 624
Class counts: [1140 3293], Class weights: [1.4856756  0.51432437]


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_47/1993662842.py:406: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


Model with DEiT stem created!
Number of parameters: 86868483
Starting training with early stopping (patience=5)...


Epoch 1/20:   0%|          | 0/277 [00:00<?, ?it/s]/tmp/ipykernel_47/1993662842.py:535: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Eval: 100%|██████████| 49/49 [00:29<00:00,  1.64it/s]


Epoch 1 finished in 212.0s
Val Acc: 89.53%, Val AUC: 0.9843, Val Seg Loss: 0.9978
✅ Saved best model (AUC improved to 0.9843)


...

Epoch 11 finished in 226.3s
Val Acc: 97.32%, Val AUC: 0.9986, Val Seg Loss: 0.9978
⏳ No improvement for 4/5 epochs (best AUC: 0.9985)


Epoch 12/20:   0%|          | 0/277 [00:00<?, ?it/s]/tmp/ipykernel_47/1993662842.py:535: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Eval: 100%|██████████| 49/49 [00:30<00:00,  1.62it/s]


Epoch 12 finished in 227.9s
Val Acc: 97.83%, Val AUC: 0.9990, Val Seg Loss: 0.9978
⏳ No improvement for 5/5 epochs (best AUC: 0.9985)
🛑 Early stopping triggered at epoch 12!

Training completed. Best val AUC: 0.9985 at epoch 7

Loading best model for final test evaluation...

FINAL TEST SET EVALUATION (with TTA)


Eval: 100%|██████████| 39/39 [00:27<00:00,  1.44it/s]


DETAILED CLASSIFICATION REPORT:
              precision    recall  f1-score   support

      NORMAL       0.96      0.89      0.93       234
   PNEUMONIA       0.94      0.98      0.96       390

    accuracy                           0.95       624
   macro avg       0.95      0.94      0.94       624
weighted avg       0.95      0.95      0.95       624

CONFUSION MATRIX:
[[209  25]
 [  8 382]]

📊 TEST RESULTS:
Test Acc: 94.71%
Test AUC: 0.9855
Test Seg Loss: 0.6404

📈 PERFORMANCE COMPARISON:
Best Val AUC: 0.9985
Test AUC:     0.9855
Gap:          0.0130

Done. Model & artifacts saved under: ./ckpt_hybrid_deit_fixed
